In [ ]:
import numpy as np
import pysm3
import pysm3.units as u
from pysm3 import utils
import healpy as hp
import fgbuster
import matplotlib.pyplot as plt
import os
import qubic
center = qubic.equ2gal(0, -57)
#center = qubic.equ2gal(0, -57)
# If there is not this command, the kernel shut down every time..
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

import warnings
warnings.filterwarnings("ignore")

nside=256

def get_coverage(fsky, nside, center_radec=[0, -57]):
    center = qubic.equ2gal(center_radec[0], center_radec[1])
    uvcenter = np.array(hp.ang2vec(center[0], center[1], lonlat=True))
    uvpix = np.array(hp.pix2vec(nside, np.arange(12*nside**2)))
    ang = np.arccos(np.dot(uvcenter, uvpix))
    indices = np.argsort(ang)
    okpix = ang < -1
    okpix[indices[0:int(fsky * 12*nside**2)]] = True
    mask = np.zeros(12*nside**2)
    mask[okpix] = 1
    return mask


covmap = get_coverage(0.03, nside)
pixok = covmap > 0

hp.mollview(covmap , title='Coverage')

In [ ]:
freqs = np.array([20., 30., 40., 85., 95., 145., 155., 220., 270.])
bandwidth = np.array([5., 9., 12., 20.4, 22.8, 31.9, 34.1, 48.4, 59.4])
dnu_nu = bandwidth/freqs
beam_fwhm = np.array([11., 72.8, 72.8, 25.5, 25.5, 22.7, 22.7, 13., 13.])
mukarcmin_TT = np.array([16.5, 9.36, 11.85, 2.02, 1.78, 3.89, 4.16, 10.15, 17.4])
mukarcmin_EE = np.array([10.87, 6.2, 7.85, 1.34, 1.18, 1.8, 1.93, 4.71, 8.08])
mukarcmin_BB = np.array([10.23, 5.85, 7.4, 1.27, 1.12, 1.76, 1.89, 4.6, 7.89])
ell_min = np.array([30, 30, 30, 30, 30, 30, 30, 30, 30])
nside = np.array([512, 512, 512, 512, 512, 512, 512, 512, 512])
edges_min = freqs * (1. - dnu_nu/2)
edges_max = freqs * (1. + dnu_nu/2)
edges = [[edges_min[i], edges_max[i]] for i in range(len(freqs))]
s4_config = {
    'nbands': len(freqs),
    'frequency': freqs,
    'depth_p': 0.5*(mukarcmin_EE + mukarcmin_BB),
    'depth_i': mukarcmin_TT,
    'depth_e': mukarcmin_EE,
    'depth_b': mukarcmin_BB,
    'fwhm': beam_fwhm,
    'bandwidth': bandwidth,
    'dnu_nu': dnu_nu,
    'ell_min': ell_min,
    'nside': nside,
    'fsky': 0.03,
    'ntubes': 12,
    'nyears': 7.,
    'edges': edges,
    'effective_fraction': np.zeros(len(freqs))+1.
            }

### QUBIC Sub-optimality : values from Louise Mousset's PhD thesis
def fct_subopt(nus, suboptimality=True):
    if suboptimality:
        subnus = [150., 220]
        subval = [1.4, 1.2]
        fct_subopt = np.poly1d(np.polyfit(subnus, subval, 1))
        return fct_subopt(nus)
    else:
        return 1

subnus = [150., 220]
subval = [1.4, 1.2]


def qubicify(config, qp_nsubs, qp_effective_fraction, suboptimality=True):
    nbands = np.sum(qp_nsubs)
    qp_config = config.copy()
    for k in qp_config.keys():
        qp_config[k]=[]
    qp_config['nbands'] = nbands
    qp_config['fsky'] = config['fsky']
    qp_config['ntubes'] = config['ntubes']
    qp_config['nyears'] = config['nyears']
    qp_config['initial_band'] = []
    
    for i in range(len(config['frequency'])):
        #print(config['edges'][i][0], config['edges'][i][-1])
        newedges = np.linspace(config['edges'][i][0], config['edges'][i][-1], qp_nsubs[i]+1)
        #print(newedges)
        newfreqs = (newedges[0:-1]+newedges[1:])/2
        newbandwidth = newedges[1:] - newedges[0:-1]
        newdnu_nu = newbandwidth / newfreqs
        newfwhm = config['fwhm'][i] * config['frequency'][i]/newfreqs
        scalefactor_noise = np.sqrt(qp_nsubs[i]) * fct_subopt(config['frequency'][i], suboptimality=suboptimality) / qp_effective_fraction[i]
        newdepth_p = config['depth_p'][i] * np.ones(qp_nsub[i]) * scalefactor_noise
        newdepth_i = config['depth_i'][i] * np.ones(qp_nsub[i]) * scalefactor_noise
        newdepth_e = config['depth_e'][i] * np.ones(qp_nsub[i]) * scalefactor_noise
        newdepth_b = config['depth_b'][i] * np.ones(qp_nsub[i]) * scalefactor_noise
        newell_min = np.ones(qp_nsub[i]) * config['ell_min'][i]
        newnside = np.ones(qp_nsub[i]) * config['nside'][i]
        neweffective_fraction = np.ones(qp_nsub[i]) * qp_effective_fraction[i]
        initial_band = np.ones(qp_nsub[i]) * config['frequency'][i]
        
        for k in range(qp_nsubs[i]):
            if qp_effective_fraction[i] != 0:
                qp_config['frequency'].append(newfreqs[k])
                qp_config['depth_p'].append(newdepth_p[k])
                qp_config['depth_i'].append(newdepth_i[k])
                qp_config['depth_e'].append(newdepth_e[k])
                qp_config['depth_b'].append(newdepth_b[k])
                qp_config['fwhm'].append(newfwhm[k])
                qp_config['bandwidth'].append(newbandwidth[k])
                qp_config['dnu_nu'].append(newdnu_nu[k])
                qp_config['ell_min'].append(newell_min[k])
                qp_config['nside'].append(newnside[k])
                
                qp_config['effective_fraction'].append(neweffective_fraction[k])
                qp_config['initial_band'].append(initial_band[k])
        for k in range(qp_nsubs[i]+1):
            if qp_effective_fraction[i] != 0:
                qp_config['edges'].append(newedges[k])
    fields = ['frequency', 'depth_p', 'depth_i', 'depth_e', 'depth_b', 'fwhm', 'bandwidth', 
              'dnu_nu', 'ell_min', 'nside', 'edges', 'effective_fraction', 'initial_band']
    for j in range(len(fields)):
        qp_config[fields[j]] = np.array(qp_config[fields[j]])
        
    return qp_config



qp_nsub = np.array([1, 1, 1, 5, 5, 5, 5, 5, 5])
qp_effective_fraction = np.array([1, 1, 1, 1, 1, 1, 1, 1, 1])
qp_config = qubicify(s4_config, qp_nsub, qp_effective_fraction)
qp_config_nosub = qubicify(s4_config, qp_nsub, qp_effective_fraction, suboptimality=False)

#Corrected depths
qp_config['depth_p'][:3] = s4_config['depth_p'][:3]
qp_config['depth_i'][:3] = s4_config['depth_i'][:3]

plt.figure(figsize=(16, 5))
plt.subplot(1,2,1)
plt.errorbar(s4_config['frequency'], s4_config['depth_p'], xerr=s4_config['bandwidth']/2, fmt='ro', label='CMB-S4')
plt.errorbar(qp_config['frequency'], qp_config['depth_p'], xerr=qp_config['bandwidth']/2, fmt='bo', label='BI')
plt.errorbar(qp_config_nosub['frequency'], qp_config_nosub['depth_p'], xerr=qp_config_nosub['bandwidth']/2, fmt='go', label='BI No Sub-optimality')
plt.xlabel('Frequency [GHz]')
plt.ylabel(r'Depth_p [$\mu$K.arcmin]')
plt.title('CMB-S4 Configuration')
plt.ylim(0, 20)
plt.legend()
plt.subplot(1,2,2)
plt.errorbar(s4_config['frequency'], s4_config['fwhm'], xerr=s4_config['bandwidth']/2, fmt='ro', label='CMB-S4')
plt.errorbar(qp_config['frequency'], qp_config['fwhm'], xerr=qp_config['bandwidth']/2, fmt='bo', label='BI')
plt.errorbar(qp_config_nosub['frequency'], qp_config_nosub['fwhm'], xerr=qp_config_nosub['bandwidth']/2, fmt='go', label='BI No Sub-optimality')
plt.xlabel('Frequency [GHz]')
plt.ylabel('FWHM [arcmin]')
plt.title('CMB-S4 Configuration')
plt.legend()
plt.ylim(0, 80)
plt.show()

In [ ]:
def create_dust_with_model2beta(nside, nus, betad0, betad1, nubreak, temp):

    # Create 353 GHz dust maps
    sky = pysm3.Sky(nside=nside, preset_strings=['d0'])
    maps_353GHz = sky.get_emission(353*u.GHz, None)*utils.bandpass_unit_conversion(353*u.GHz,None, u.uK_CMB)

    comp2b=[fgbuster.component_model.Dust_2b(nu0=353)]

    A2b = fgbuster.MixingMatrix(*comp2b)
    A2b_ev = A2b.evaluator(nus)
    A2b_maxL = A2b_ev([betad0, betad1, nubreak, temp])

    new_dust_map=np.zeros(((len(nus), 3, 12*nside**2)))
    for i in range(len(nus)):
        new_dust_map[i]=A2b_maxL[i, 0]*maps_353GHz

    return new_dust_map

def go_to_nside_target(param_est, nside_param_out, config, sky_conf, fix_temp):
    
    nside_param_est_in=hp.pixelfunc.npix2nside(param_est.shape[1])
    #print('Nside in : ', nside_param_est_in)
    
    param_est_nsideout=np.zeros((param_est.shape[0], 12*nside_param_out**2))
    for i in range(param_est.shape[0]):
        param_est_nsideout[i] = hp.pixelfunc.ud_grade(param_est[i], nside_param_out)
        
    nside_param_est_out=hp.pixelfunc.npix2nside(param_est_nsideout.shape[1])
    #print('Nside out : ', nside_param_est_out)
    
    
    # Dust reconstruction
    
    covmap = get_coverage(0.03, nside_param_out)
    pixok = covmap>0
    
    
    
    
    # Create components from beta estimated
    
    preset_strings = [sky_conf[i:i+1] for i in range(0, len(sky_conf), 1)]
    
    comp=np.zeros((((len(preset_strings), len(config['frequency']), 3, 12*nside_param_out**2))))
    k=0
    for i in preset_strings:
        if i == 'd':
            
            
            if param_est.shape[0] == 5:
                map_beta0=param_est_nsideout[0]
                map_beta1=param_est_nsideout[1]
                map_nubreak=param_est_nsideout[2]
                map_temp=param_est_nsideout[3]
            else:
                map_beta0=param_est_nsideout[0]
                map_beta1=param_est_nsideout[1]
                map_nubreak=param_est_nsideout[2]
                map_temp=np.ones(12*nside_param_out**2)*20
    
            comp[k]=give_me_dust_d1_estimated(config['frequency'], map_beta0, map_beta1, map_nubreak, 
                         map_temp, covmap, nside_param_out, fix_temp=fix_temp)
            k+=1
    
        elif i == 's':
            
            map_betapl=param_est_nsideout[-1]
            comp[k]=give_me_sync_d1_modified(config['frequency'], map_betapl, covmap, nside_param_out)
            k+=1
        else:
            pass
        
    return comp, param_est_nsideout


def get_comp_for_fgb(nu0, model, fix_temp, x0):
    comp=[fgbuster.component_model.CMB()]
    if model == '1b':
        if fix_temp is not None :
            comp.append(fgbuster.component_model.Dust(nu0=nu0, temp=fix_temp))
            comp[1].defaults=x0
        else:
            comp.append(fgbuster.component_model.Dust(nu0=nu0))
            comp[1].defaults=x0
    elif model == '2b':
        if fix_temp is not None :
            comp.append(fgbuster.component_model.Dust_2b(nu0=nu0, temp=fix_temp))
            comp[1].defaults=x0
        else:
            comp.append(fgbuster.component_model.Dust_2b(nu0=nu0))
            comp[1].defaults=x0
    else:
        raise TypeError('Not the good model')
        
    comp.append(fgbuster.component_model.Synchrotron(nu0=145))
    comp[2].defaults=[-3]
      
    return comp


def get_comp_from_MixingMatrix(r, comp, instr, data, delta_beta, covmap, model, noise, nside):
    
    """
    
    This function estimate components from MixingMatrix of fgbuster with estimated parameters
    
    """
    
    pixok=covmap>0
    
    # Define Mixing Matrix from FGB 
    A=fgbuster.mixingmatrix.MixingMatrix(*comp)
    A_ev=A.evaluator(np.array(instr.frequency))
    A_maxL=A_ev(np.array(r.x))

    if noise:
        invN = np.diag(hp.nside2resol(nside, arcmin=True) / (instr.depth_p))**2
        maps_separe=fgbuster.algebra.Wd(A_maxL, data.T, invN=invN).T
    else:
        maps_separe=fgbuster.algebra.Wd(A_maxL, data.T).T

    maps_separe[:, :, ~pixok]=hp.UNSEEN
    
    return maps_separe

In [ ]:
import qubicplus
from importlib import reload
reload(qubicplus)

def ParamCompSep(config, nside, skyconfig, model, noise=True, delta_beta=0.05, fix_temp=None, nubreak=260, x0=[1.54, 1.54, 140, 22]):

    covmap = get_coverage(0.03, nside)
    pixok = covmap>0
    
    if len(config['frequency']) == 9 :
        name='CMBS4'
    elif len(config['frequency']) == 33 :
        name='Qubic+'
    else:
        raise TypeError('Choose the right instrument')
    
    print('Create maps')
    if noise:
        inputs, _, _, _ = qubicplus.BImaps(skyconfig, config, nside=nside).getskymaps(
                                      same_resol=0, 
                                      verbose=False, 
                                      coverage=covmap, 
                                      noise=True, 
                                      signoise=1.,
                                      beta=[1.54-delta_beta, 1.54+delta_beta, nubreak],
                                      fix_temp=fix_temp)
        
    else:
        _, inputs, _, _ = qubicplus.BImaps(skyconfig, config, nside=nside).getskymaps(
                                      same_resol=0, 
                                      verbose=False, 
                                      coverage=covmap, 
                                      noise=True, 
                                      signoise=1.,
                                      beta=[1.54-delta_beta, 1.54+delta_beta, nubreak],
                                      fix_temp=fix_temp)
        
        
        
    ###### Preparation for COMPSEP ######
    
    

    print('Define instrument')
    instr = fgbuster.get_instrument(name)
    instr.frequency=config['frequency']
    instr.depth_i=config['depth_i']
    instr.depth_p=config['depth_p']
    instr.fwhm=np.zeros(len(config['frequency']))
    
    print('Define components')
    comp=get_comp_for_fgb(nu0=145, model=model, fix_temp=fix_temp, x0=x0)

    print('##### COMP SEP #####')
    
    options={'maxiter':100}
    tol=1e-18
    bnds=((0, None), (0, None), (0, None), (0, None), (None, 0))
    r=fgbuster.basic_comp_sep(comp, instr, inputs[:, :, pixok], tol=tol, method='TNC', 
                                                                             options=options, bounds=bnds)
    print('Done! \n')
    print(r.x)
    
    components=get_comp_from_MixingMatrix(r, comp, instr, inputs, delta_beta, covmap, model, noise, nside)
    
    return components, r

# You can play here with the code

In [ ]:
'''
nside=256                            # NSIDE of input maps
db=0.1                               # Deviation from true beta maps
fix_temp=None                        # If you don't want to fix temp, put None
noise=False
nubreak=100
x0=[1.54, 1.54, 150, 22]

components_s4, r_s4 = ParamCompSep(s4_config,
                                   skyconfig={'cmb':42,'dust':'d02b', 'synchrotron':'s0'},
                                   model='2b',
                                   noise=noise,
                                   delta_beta=db,
                                   fix_temp=fix_temp,
                                   nside=nside,
                                   nubreak=nubreak,
                                   x0=x0)

components_bi, r_bi = ParamCompSep(qp_config,
                                   skyconfig={'cmb':42,'dust':'d02b', 'synchrotron':'s0'},
                                   model='2b',
                                   noise=noise,
                                   delta_beta=db,
                                   fix_temp=fix_temp,
                                   nside=nside,
                                   nubreak=nubreak,
                                   x0=x0)
'''

In [ ]:
'''
istk=1
ind=5

plt.figure(figsize=(15, 8))

hp.gnomview(truedust[ind, istk], rot=center, reso=15, cmap='jet', sub=(2, 5, 1), title='True')
hp.gnomview(components_s4[1, istk], rot=center, reso=15, cmap='jet', sub=(2, 5, 2), title='S4')
hp.gnomview(components_bi[1, istk], rot=center, reso=15, cmap='jet', sub=(2, 5, 3), title='BI')


s4=truedust[ind, istk]-components_s4[1, istk]
bi=truedust[ind, istk]-components_bi[1, istk]
s4[~pixok]=hp.UNSEEN
bi[~pixok]=hp.UNSEEN

min=-np.max(s4[pixok])
max=np.max(s4[pixok])

hp.gnomview(s4, rot=center, reso=15, cmap='jet', sub=(2, 5, 4), title='RMS = {:.6g}'.format(np.std(s4[pixok])), min=min, max=max)
hp.gnomview(bi, rot=center, reso=15, cmap='jet', sub=(2, 5, 5), title='RMS = {:.6g}'.format(np.std(bi[pixok])), min=min, max=max)

hp.gnomview(truecmb[ind, istk], rot=center, reso=15, cmap='jet', sub=(2, 5, 6), title='True')
hp.gnomview(components_s4[0, istk], rot=center, reso=15, cmap='jet', sub=(2, 5, 7), title='S4')
hp.gnomview(components_bi[0, istk], rot=center, reso=15, cmap='jet', sub=(2, 5, 8), title='BI')

s4=truecmb[ind, istk]-components_s4[0, istk]
bi=truecmb[ind, istk]-components_bi[0, istk]
s4[~pixok]=hp.UNSEEN
bi[~pixok]=hp.UNSEEN


hp.gnomview(s4, rot=center, reso=15, cmap='jet', sub=(2, 5, 9),
            title='True - S4 -> dr : {:.5g}'.format((np.std(s4[pixok])/0.345)**2))
hp.gnomview(bi, rot=center, reso=15, cmap='jet', sub=(2, 5, 10),
            title='True - BI -> dr : {:.5g}'.format((np.std(bi[pixok])/0.345)**2))


plt.show()
'''

In [ ]:
def get_good_config(config, prop):
    config1=config[0]
    config2=config[1]
    nus=np.array(list(config1['frequency'])+list(config2['frequency']))
    depth1_i=config1['depth_i']/(np.sqrt(prop[0]))
    depth1_p=config1['depth_p']/(np.sqrt(prop[0]))
    depth2_i=config2['depth_i']/(np.sqrt(prop[1]))
    depth2_p=config2['depth_p']/(np.sqrt(prop[1]))

    depth_i=np.array(list(depth1_i)+list(depth2_i))
    depth_p=np.array(list(depth1_p)+list(depth2_p))
    fwhm=np.zeros(42)
    
    if prop[0] == 1 :
        depth_i=config1['depth_i']
        depth_p=config1['depth_p']
        nus=config1['frequency']
        fwhm=np.zeros(9)
    elif prop[1] == 1:
        depth_i=config2['depth_i']
        depth_p=config2['depth_p']
        nus=config2['frequency']
        fwhm=np.zeros(33)
    else:
        pass
    
    

    return nus, depth_i, depth_p, fwhm

def get_cov_for_weighted(n_freq, depths_i, depths_p, coverage, nside=256):
    npix=12*nside**2
    ind=coverage > 0
    
    noise_cov = np.ones(((n_freq, 3, npix)))
    
    for i in range(n_freq):
        noise_cov[i, 0] = np.ones(npix)*1./depths_i[i]**2
        noise_cov[i, 1] = np.ones(npix)*1./depths_p[i]**2
        noise_cov[i, 2] = np.ones(npix)*1./depths_p[i]**2
    
    noise_cov[:, :, ~ind]=hp.UNSEEN
    
    return noise_cov

def ParamCompSep(config, prop, nside, skyconfig, model, noise=True, delta_beta=0.05, fix_temp=None, 
                 nubreak=260, x0=[], weighted=False, reduce_noise=1):

    covmap = get_coverage(0.03, nside)
    pixok = covmap>0



    print('Create maps')
    if noise:
        inputs, _, _ = qubicplus.combinedmaps(skyconfig, config, nside=nside, prop=prop).getskymaps(
                                      verbose=False,
                                      coverage=covmap,
                                      noise=True,
                                      beta=[1.54-delta_beta, 1.54+delta_beta, nubreak],
                                      fix_temp=fix_temp)

    else:
        _, inputs, _ = qubicplus.combinedmaps(skyconfig, config, nside=nside, prop=prop).getskymaps(
                                      verbose=False,
                                      coverage=covmap,
                                      noise=True,
                                      beta=[1.54-delta_beta, 1.54+delta_beta, nubreak],
                                      fix_temp=fix_temp)



    ###### Preparation for COMPSEP ######


    nus, depth_i, depth_p, fwhm = get_good_config(config, prop)
    
    #if noise is False :
    #    depth_i/=10000
    #    depth_p/=10000
    #    print('Reduce depths')
    
    if prop[0]==1:
        name='CMBS4'
        n_freq=9
    elif prop[1]==1:
        name='Qubic+'
        n_freq=33
    else:
        name='CMBS4BI'
        n_freq=42
    
    print('Define instrument')
    instr = fgbuster.get_instrument(name)
    instr.frequency=nus
    instr.depth_i=depth_i
    instr.depth_p=depth_p
    instr.fwhm=fwhm

    print('Define components')
    comp=get_comp_for_fgb(nu0=145, model=model, fix_temp=fix_temp, x0=x0)

    print('##### COMP SEP #####')

    options={'maxiter':10000000, 'eps':1e-8} 
    tol=1e-18
#     bnds=((0, None), (0, None), (0, None), (0, None), (None, 0))
    bnds=((0, None), (0, None), (0, None), (0., 40.), (None, 0))
    
    if weighted:
        print('wei')
        noise_cov=get_cov_for_weighted(n_freq, depth_i, depth_p, covmap)
        r=fgbuster.weighted_comp_sep(comp, instr, inputs[:, :, :], cov=noise_cov, tol=tol, method='TNC',
                                                                             options=option, bounds=bnds)
        
    else:
        r=fgbuster.basic_comp_sep(comp, instr, inputs[:, :, pixok], tol=tol, method='TNC',
                                                                             options=options, bounds=bnds)
    
    print('Done! \n')
    print(r.x)

    components=get_comp_from_MixingMatrix(r, comp, instr, inputs, delta_beta, covmap, model, noise, nside)
    return components, r

# Single-shot version (large flukes)

In [ ]:
from importlib import reload
reload(fgbuster)
reload(qubicplus)

ntest = 3
#props = np.array([0.00001, 0.1, 0.9, 0.99999])
props = np.linspace(0, 1, ntest)

#### Scale noise
noise_reduction = 1
my_s4_config = s4_config.copy()
# my_qp_config = qp_config.copy()
my_qp_config = qp_config_nosub.copy()

my_s4_config['depth_i'] /= noise_reduction
my_s4_config['depth_p'] /= noise_reduction
my_qp_config['depth_i'] /= noise_reduction
my_qp_config['depth_p'] /= noise_reduction

x_est=np.zeros((ntest, 5))
rms_est=np.zeros(ntest)
seed=42
fix_temp=None
db=0.05
noise=True
nside=256
nubreak=100
model='2b'
x0=[1.54, 1.54, nubreak, 20]

covmap=get_coverage(0.03, 256)
pixok=covmap>0
fix_temp=None

In [ ]:
_, truecmb, _ = qubicplus.BImaps({'cmb':42}, my_s4_config, nside=nside).getskymaps(
                                      same_resol=0,
                                      verbose=False,
                                      coverage=covmap,
                                      noise=True,
                                      signoise=1.,
                                      beta=[],
                                      fix_temp=fix_temp)

_, truedust, _ = qubicplus.BImaps({'dust':'d02b'}, my_s4_config, nside=nside).getskymaps(
                                      same_resol=0,
                                      verbose=False,
                                      coverage=covmap,
                                      noise=True,
                                      signoise=1.,
                                      beta=[1.54-db, 1.54+db, nubreak],
                                      fix_temp=fix_temp)

In [ ]:
for i in range(ntest):
    print('Props {}/{}'.format(i,ntest))
    BIprop=props[i]
    S4prop=1-BIprop
    myx0 = x0.copy()
    myx0[2] = 200.#(np.random.rand()-0.5)*100+nubreak
    components_comb_1, r_comb_1 = ParamCompSep([my_s4_config, my_qp_config],
                                   prop=[S4prop, BIprop],
                                   skyconfig={'cmb':seed, 'dust':'d02b', 'synchrotron':'s0'},
                                   model=model,
                                   noise=noise,
                                   delta_beta=db,
                                   fix_temp=fix_temp,
                                   nside=nside,
                                   nubreak=nubreak,
                                   x0=myx0,
                                   weighted=False)
    print()
    print('#########')
    print('Message from fgb -> ', r_comb_1.message)
    print('# of function evaluations -> ', r_comb_1.nfev)
    print('#########')
    print()

    x_est[i, :]=r_comb_1.x
    rms_est[i]=np.std(truecmb[0, 1, pixok]-components_comb_1[0, 1, pixok])
    print('RMS -> ', the_rms_est[i])
    print('')
    print('')
    print('')
    

In [ ]:
plt.figure(figsize=(30, 6))
truth=[1.49, 1.59, 100, 20, -3]
truth_label=[r'$\beta_0$', r'$\beta_1$', r'$nu_{break}$', r'$T$', r'$\beta_s$']
for i in range(5):
    plt.subplot(1, 5, i+1)
    plt.plot(props, x_est[:, i])
    plt.axhline(truth[i], ls='--', color='black')
    plt.ylabel(truth_label[i], fontsize=30)
plt.show()

plt.figure()
plt.plot(props, rms_est)
yscale('log')
plt.ylabel('RMS CMB')
plt.xlabel('Bi Proportion')
plt.title('Max-min = {0:6.4f}'.format(np.max(rms_est)-np.min(rms_est)))
plt.show()


# Monte-Carlo version (long)

In [ ]:
from importlib import reload
reload(fgbuster)
reload(qubicplus)

ntest = 5
#props = np.array([0.00001, 0.1, 0.9, 0.99999])
props = np.linspace(0, 1, ntest)

#### Scale noise
noise_reduction = 1
my_s4_config = s4_config.copy()
# my_qp_config = qp_config.copy()
my_qp_config = qp_config_nosub.copy()

my_s4_config['depth_i'] /= noise_reduction
my_s4_config['depth_p'] /= noise_reduction
my_qp_config['depth_i'] /= noise_reduction
my_qp_config['depth_p'] /= noise_reduction

x_est=np.zeros((ntest, 5))
rms_est=np.zeros(ntest)
seed=42
fix_temp=None
db=0.05
noise=True
nside=256
nubreak=100
model='2b'
x0=[1.54, 1.54, nubreak, 20]

covmap=get_coverage(0.03, 256)
pixok=covmap>0
fix_temp=None

In [ ]:
_, truecmb, _ = qubicplus.BImaps({'cmb':42}, my_s4_config, nside=nside).getskymaps(
                                      same_resol=0,
                                      verbose=False,
                                      coverage=covmap,
                                      noise=True,
                                      signoise=1.,
                                      beta=[],
                                      fix_temp=fix_temp)

_, truedust, _ = qubicplus.BImaps({'dust':'d02b'}, my_s4_config, nside=nside).getskymaps(
                                      same_resol=0,
                                      verbose=False,
                                      coverage=covmap,
                                      noise=True,
                                      signoise=1.,
                                      beta=[1.54-db, 1.54+db, nubreak],
                                      fix_temp=fix_temp)

In [ ]:
nbmc = 10
x_est=np.zeros((ntest, 5))
rms_est=np.zeros(ntest)
sig_x_est=np.zeros((ntest, 5))
sig_rms_est=np.zeros(ntest)

for i in range(ntest):
    the_x_est=np.zeros((nbmc, 5))
    the_rms_est=np.zeros(nbmc)
    for j in range(nbmc):
        print('Props {}/{} - MC {}/{}'.format(i,ntest,j,nbmc))
        BIprop=props[i]
        S4prop=1-BIprop
        myx0 = x0.copy()
        myx0[2] = 140#(np.random.rand()-0.5)*40+nubreak
        components_comb_1, r_comb_1 = ParamCompSep([my_s4_config, my_qp_config],
                                       prop=[S4prop, BIprop],
                                       skyconfig={'cmb':seed, 'dust':'d02b', 'synchrotron':'s0'},
                                       model=model,
                                       noise=noise,
                                       delta_beta=db,
                                       fix_temp=fix_temp,
                                       nside=nside,
                                       nubreak=nubreak,
                                       x0=myx0,
                                       weighted=False)
        print()
        print('#########')
        print('Message from fgb -> ', r_comb_1.message)
        print('# of function evaluations -> ', r_comb_1.nfev)
        print('#########')
        print()

        the_x_est[j, :]=r_comb_1.x
        the_rms_est[j]=np.std(truecmb[0, 1, pixok]-components_comb_1[0, 1, pixok])
        print('RMS -> ', the_rms_est[j])
        print('')
        print('')
        print('')
    x_est[i,:] = np.mean(the_x_est, axis=0)    
    sig_x_est[i,:] = np.std(the_x_est, axis=0)
    rms_est[i] = np.mean(the_rms_est)
    sig_rms_est[i] = np.std(the_rms_est)
    

In [ ]:
plt.figure(figsize=(30, 6))
truth=[1.49, 1.59, 100, 20, -3]
truth_label=[r'$\beta_0$', r'$\beta_1$', r'$nu_{break}$', r'$T$', r'$\beta_s$']
for i in range(5):
    plt.subplot(1, 5, i+1)
    plt.errorbar(props, x_est[:, i], yerr=sig_x_est[:, i], fmt='ro-')
    plt.axhline(truth[i], ls='--', color='black')
    plt.ylabel(truth_label[i], fontsize=30)
plt.show()

plt.figure()
plt.errorbar(props, rms_est, yerr=sig_rms_est, fmt='ro-')
yscale('log')
plt.ylabel('RMS CMB')
plt.xlabel('Bi Proportion')
plt.show()

print(x0)

In [ ]:
1.02e-1-9.95e-2

In [ ]:
np.savez('sim_supbopt_w_0.5_0.npz', props=props, rms_est=rms_est, x_est=x_est)
data = np.load('sim_nosupbopt_w_0.5_0.npz')
print(data['props'])
print(data['rms_est'])
print(data['x_est'])

In [ ]:
data = np.load('sim_nosupbopt_w_0.5_0.npz')
print(data['props'])
print(data['rms_est'])
print(data['x_est'])

plt.figure(figsize=(30, 6))
truth=[1.49, 1.59, 100, 20, -3]
truth_label=[r'$\beta_0$', r'$\beta_1$', r'$nu_{break}$', r'$T$', r'$\beta_s$']
for i in range(5):
    plt.subplot(1, 5, i+1)
    plt.plot(props, x_est[:, i])
    plt.axhline(truth[i], ls='--', color='black')
    plt.ylabel(truth_label[i], fontsize=30)
plt.show()

plt.figure()
plt.plot(props, rms_est)
yscale('log')
plt.ylabel('RMS CMB')
plt.xlabel('Bi Proportion')
plt.show()
